In [91]:
import pandas as pd
import numpy as np

# graphics packages
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

%matplotlib inline

## Problem 1

### Question 1: You should download and combine data from the open data portal for 2017 about the following 311 requests: Graffiti Removal, Vacant and Abandoned Buildings Reported, Alley Lights Out

**Step 1: understanding the data schema** The first thing that I wanted to do was read in the data and identify what fields I was working with. I also knew that I would have to fill nulls in. 

In [92]:
# import csv files
graffiti = pd.read_csv('./311_Service_Requests_-_Graffiti_Removal.csv').fillna('unknown')
vacant = pd.read_csv('./311_Service_Requests_-_Vacant_and_Abandoned_Buildings_Reported.csv').fillna('unknown')
alleys = pd.read_csv('./311_Service_Requests_-_Alley_Lights_Out.csv').fillna('unknown')

In [93]:
print(graffiti.columns)
print(vacant.columns)
print(alleys.columns)

Index(['Creation Date', 'Status', 'Completion Date', 'Service Request Number',
       'Type of Service Request', 'What Type of Surface is the Graffiti on?',
       'Where is the Graffiti located?', 'Street Address', 'ZIP Code',
       'X Coordinate', 'Y Coordinate', 'Ward', 'Police District',
       'Community Area', 'SSA', 'Latitude', 'Longitude', 'Location'],
      dtype='object')
Index(['SERVICE REQUEST TYPE', 'SERVICE REQUEST NUMBER',
       'DATE SERVICE REQUEST WAS RECEIVED',
       'LOCATION OF BUILDING ON THE LOT (IF GARAGE, CHANGE TYPE CODE TO BGD).',
       'IS THE BUILDING DANGEROUS OR HAZARDOUS?',
       'IS BUILDING OPEN OR BOARDED?',
       'IF THE BUILDING IS OPEN, WHERE IS THE ENTRY POINT?',
       'IS THE BUILDING CURRENTLY VACANT OR OCCUPIED?',
       'IS THE BUILDING VACANT DUE TO FIRE?',
       'ANY PEOPLE USING PROPERTY? (HOMELESS, CHILDEN, GANGS)',
       'ADDRESS STREET NUMBER', 'ADDRESS STREET DIRECTION',
       'ADDRESS STREET NAME', 'ADDRESS STREET SUFFIX', 'Z

In [94]:
# this is a sample exploratory analysis line -- it was to show me how to collapse values
vacant['IS BUILDING OPEN OR BOARDED?'].unique()

array(['unknown', 'Building is Open / Unsecure', 'Building is Boarded Up',
       'Open', 'Boarded'], dtype=object)

**Step 2: Preliminary data cleaning** I then wanted to clean the data up a little and standardize it so that all of the frames could be easily concatenated into one large dataframe. I did this per dataframe so that I could reduce errors. 

In [103]:
# create type and subtype columns for each dataframe
vacant['reqtype'] = vacant['SERVICE REQUEST TYPE']

# we care more that it is definitely boarded than if it is not
vacant['boarded'] = np.where(vacant['IS BUILDING OPEN OR BOARDED?'].str.match('Boarded'), 'boarded', 'not_boarded')

# fire, in use vacancy (dangerous and hazardous always na)
vacant['fire'] = np.where(vacant['IS THE BUILDING VACANT DUE TO FIRE?'], 'fire_vacancy', 'not_fire')
vacant['in_use'] = np.where(vacant['ANY PEOPLE USING PROPERTY? (HOMELESS, CHILDEN, GANGS)'], 'in_use', 'not_in_use')

# vacancy subtype
vacant['subtype'] = vacant[['boarded', 'fire', 'in_use']].apply(lambda x: "_".join(x), axis=1)

# create a status indicator
vacant['Status'] = np.where(vacant['DATE SERVICE REQUEST WAS RECEIVED'].isnull(), 'Open', 'Completed')
vacant['response time'] =  ['Not_applicable'] * vacant['SERVICE REQUEST TYPE'].count()

# now we do the same thing for the other two dfs
graffiti['reqtype'] = graffiti['Type of Service Request']
graffiti['subtype'] = graffiti[['What Type of Surface is the Graffiti on?',
                                'Where is the Graffiti located?']].apply(lambda x: "_".join(x), axis=1)
graffiti['Completion Date'] = pd.as_datetime(graffiti['Completion Date']) 
alleys['reqtype'] = alleys['Type of Service Request']
alleys['subtype'] = alleys['Type of Service Request']

AttributeError: module 'pandas' has no attribute 'as_datetime'

**Step 3: Data merging** The final step in ensuring that the dataframes are ready for the merge is selecting the columns to retain and setting new indices. Then, we merge the frames. 

In [96]:
perm_cols = ['Status', 'Service Request Number', 'Creation Date', 
             'Completion Date', 'Police District', 'Community Area', 'Latitude',
       'Longitude','reqtype', 'subtype']

vacant_merge = vacant[[ 'Status', 'SERVICE REQUEST NUMBER', 'DATE SERVICE REQUEST WAS RECEIVED', 'completed date',
                       'Police District',
                       'Community Area', 'LATITUDE', 'LONGITUDE', 'reqtype','subtype']]
vacant_merge.columns = perm_cols
vacant_merge.set_index('Service Request Number')

alleys_merge = alleys[perm_cols]
alleys_merge.set_index('Service Request Number')

graffiti_merge = graffiti[perm_cols]
graffiti_merge.set_index('Service Request Number')

all_311 = pd.concat([vacant_merge, alleys_merge, graffiti_merge]).set_index('Service Request Number')

At this point, all of the data is combined into one dataframe, named **all_311**

### Question 2: Generate summary statistics for these requests including but not limited to number of requests of each type (and subtype within each of the types above) over time, by neighborhood, response time by the city. Please use a combination of tables and graphs to present these summary stats.

In [97]:
# police districts crosswalk -- for labelling 
districts = {
    1: "central",
    2: 'wentworth',
    3: 'grand crossing', 
    4: 'south chicago', 
    5: 'calumet', 
    6: 'gresham', 
    7: 'englewood', 
    8: 'chicago lawn', 
    9: 'deering', 
    10: 'ogden', 
    11: 'harrison', 
    12: 'near west', 
    14: 'shakespeare', 
    15: 'austin', 
    16: 'jefferson park', 
    17: 'albany park', 
    18: 'near north', 
    19: 'town hall', 
    20: 'lincoln',
    22: 'morgan park',
    24: 'rogers park',
    25: 'grand central'
}

all_311['neighborhood'] = \
    all_311.apply(lambda x: districts.get(x['Police District'], 'Unknown'), axis=1)

In [98]:
# first, I just want to get a sense of how many of these complaints are open / completed
all_311.groupby(['reqtype', 'Status']).size()

reqtype                    Status         
Alley Light Out            Completed          107346
                           Completed - Dup     83715
                           Open                 4260
                           Open - Dup           4269
Graffiti Removal           Completed          887889
                           Completed - Dup     85923
                           Open                  128
                           Open - Dup             15
Vacant/Abandoned Building  Completed           62125
dtype: int64

In [99]:
# make a heat map by neighborhood for the number of complaints
grouped = all_311.groupby(['neighborhood', 'reqtype']).size().unstack().reset_index()

sns.barplot(x="neighborhood", y="Vacant/Abandoned Building", ax=ax, data=grouped)

In [100]:
grouped.columns

Index(['neighborhood', 'Alley Light Out', 'Graffiti Removal',
       'Vacant/Abandoned Building'],
      dtype='object', name='reqtype')

In [101]:
# i know a fair amount about police -- District 13 was dissolved when the police department was redistricted. This could explain why complaints are low... but in any case...


Based on these summary statistics, tell me 5 interesting things you learned
(about Chicago and the different neighborhoods) using the 311 data

In [102]:
vacant.columns

Index(['SERVICE REQUEST TYPE', 'SERVICE REQUEST NUMBER',
       'DATE SERVICE REQUEST WAS RECEIVED',
       'LOCATION OF BUILDING ON THE LOT (IF GARAGE, CHANGE TYPE CODE TO BGD).',
       'IS THE BUILDING DANGEROUS OR HAZARDOUS?',
       'IS BUILDING OPEN OR BOARDED?',
       'IF THE BUILDING IS OPEN, WHERE IS THE ENTRY POINT?',
       'IS THE BUILDING CURRENTLY VACANT OR OCCUPIED?',
       'IS THE BUILDING VACANT DUE TO FIRE?',
       'ANY PEOPLE USING PROPERTY? (HOMELESS, CHILDEN, GANGS)',
       'ADDRESS STREET NUMBER', 'ADDRESS STREET DIRECTION',
       'ADDRESS STREET NAME', 'ADDRESS STREET SUFFIX', 'ZIP CODE',
       'X COORDINATE', 'Y COORDINATE', 'Ward', 'Police District',
       'Community Area', 'LATITUDE', 'LONGITUDE', 'Location', 'reqtype',
       'boarded', 'fire', 'in_use', 'subtype', 'Status', 'completed date'],
      dtype='object')